<a href="https://colab.research.google.com/github/eyesonaplate/ITB-2022/blob/main/w15_%EA%B8%B0%EB%A7%90%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 필수 실행 코드

In [ ]:
#한글 깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf 

#해당 코드 실행 후 런타임을 다시 실행하면 한글이 깨지지 않음

In [1]:
#패키지 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#폰트 설정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [3]:
#파일 경로 설정
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **데이터 전처리0. charts.csv 전처리**

charts.csv: 1958년 8월 4일부터 2021년 11월 6일까지 매 주차 스포티파이 Top100 차트에 랭크된 곡에 대한 정보를 포함하고 있다. top100 아티스트와 곡 정보를 추출한 뒤 다른 데이터와 연결해 top100 차트인 여부를 판단하는 새로운 변수 top100 만들 때 활용하고자 한다

*   week: 해당 월 주차 수
*   date: 날짜
*   rank: 순위
*   song: 곡명
*   artist: 아티스트명
*   peak_rank: 최고 순위
*   weeks-on-board: Top100 차트인 기간


In [75]:
#데이터 불러오기
charts = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ITB 2022/charts_new.csv",  encoding='cp949')

In [34]:
#데이터 탐색하기
display(charts.head())

,week,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,1,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,1,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,1,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,1,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [35]:
#데이터 탐색하기2
charts.describe

<bound method NDFrame.describe of         week        date  rank                                 song  \
0          1  2021-11-06     1                           Easy On Me   
1          1  2021-11-06     2                                 Stay   
2          1  2021-11-06     3                        Industry Baby   
3          1  2021-11-06     4                           Fancy Like   
4          1  2021-11-06     5                           Bad Habits   
...      ...         ...   ...                                  ...   
330082     2  1958-08-04    96                        Over And Over   
330083     2  1958-08-04    97                     I Believe In You   
330084     2  1958-08-04    98                      Little Serenade   
330085     2  1958-08-04    99  I'll Get By (As Long As I Have You)   
330086     2  1958-08-04   100                                 Judy   

                               artist  last-week  peak-rank  weeks-on-board  
0                               Ade

**0-1) 적정한 기간 설정: 2000년 1월 ~ 2019년 12월까지 10년간의 데이터 추출**

In [77]:
#date를 인덱스로 설정
charts = charts.set_index('date')

#기간에 맞는 행만 추출해 charts에 저장
charts = charts['2019-12-31':'2000-01-01']

#artist를 기준으로 중복되는 행 제거
charts.drop_duplicates(['artist'])

,week,rank,song,artist,last-week,peak-rank,weeks-on-board
date,,,,,,,
2019-12-28,5,1,All I Want For Christmas Is You,Mariah Carey,1.0,1,36
2019-12-28,5,2,Rockin' Around The Christmas Tree,Brenda Lee,3.0,2,31
2019-12-28,5,3,Circles,Post Malone,2.0,1,16
2019-12-28,5,4,Roxanne,Arizona Zervas,6.0,4,7
2019-12-28,5,5,Futsal Shuffle 2020,Lil Uzi Vert,NaN,5,1
...,...,...,...,...,...,...,...
2000-01-15,3,61,Heartbreaker,Mariah Carey Featuring Jay-Z,66.0,1,20
2000-01-15,3,78,15 Minutes,Marc Nelson,67.0,27,16
2000-01-15,3,92,A Country Boy Can Survive (Y2K Version),Chad Brock With Hank Williams Jr. & George Jones,75.0,75,3


**0-2) 데이터 분석에 사용할 변수만 추출: song, artist,rank**

In [78]:
#칼럼 추출하기
charts = charts.loc[:,['song','artist','rank']]

display(charts.head())

,song,artist,rank
date,,,
2019-12-28,All I Want For Christmas Is You,Mariah Carey,1
2019-12-28,Rockin' Around The Christmas Tree,Brenda Lee,2
2019-12-28,Circles,Post Malone,3
2019-12-28,Roxanne,Arizona Zervas,4
2019-12-28,Futsal Shuffle 2020,Lil Uzi Vert,5


### **데이터 전처리1.spotify_artist 데이터 만들기**

feature_corel_100.csv는 12904명의 스포티파이 아티스트에 대한 통계적 정보를 보여준다. charts 데이터와 연결해 해당 아티스트가 2000~2019년 스포티파이 top100에 포함된 적이 있는 지를 판단하고자 한다.


*   name: 아티스트명
*   popularity: 아티스트 좋아요 수
*   followers: 아티스트 팔로워 수
*   num_release: 발매한 노래 수
*   pop_mean:발매한 노래의 좋아요 수 평균
*   fol_mean:?
*   rel_mean:?



In [79]:
#feature_corel_100.csv파일 불러오기
feature = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ITB 2022/feature_corel_100.csv")

In [ ]:
#데이터 확인
print(feature)

**1-1) 데이터 분석에 사용할 변수만 추출: name, popularity, followers, num_release, pop_mean, fol_mean, rel_mean**

In [81]:
feature = feature.loc[:,['name','popularity','followers','num_release','pop_mean','rel_mean','fol_mean']]

**1-2) 변수명 바꾸기**

In [82]:
#name을 artist로 변수명 바꾸기
feature.rename(columns = {'name':'artist'},inplace=True)
display(feature.head())

,artist,popularity,followers,num_release,pop_mean,rel_mean,fol_mean
0,Drake,98.0,61377383.0,124,81.377358,53.075472,1.100285e+07
1,Ed Sheeran,95.0,93327682.0,34,80.035714,44.035714,1.472265e+07
2,Ariana Grande,93.0,76626333.0,94,77.650000,51.900000,1.000831e+07
3,Bad Bunny,100.0,44516743.0,30,75.628571,23.142857,8.425305e+06
4,The Weeknd,98.0,42275433.0,108,81.928571,53.178571,1.363645e+07


**1-3) charts와 feature를 합친 새로운 데이터프레임 spotify_artist 만들기**


In [88]:
#charts 데이터 복사하기
charts_1 = charts.copy()

#artist를 기준으로 데이터 합치기
spotify_artist = pd.merge(feature,charts_1,how='left',on='artist')
display(spotify_artist)

,artist,popularity,followers,num_release,pop_mean,rel_mean,fol_mean,song,rank
0,Drake,98.0,61377383.0,124,81.377358,53.075472,1.100285e+07,Behind Barz,83.0
1,Drake,98.0,61377383.0,124,81.377358,53.075472,1.100285e+07,Behind Barz,75.0
2,Drake,98.0,61377383.0,124,81.377358,53.075472,1.100285e+07,Trust Issues,58.0
3,Drake,98.0,61377383.0,124,81.377358,53.075472,1.100285e+07,How About Now,60.0
4,Drake,98.0,61377383.0,124,81.377358,53.075472,1.100285e+07,The Motion,61.0
...,...,...,...,...,...,...,...,...,...
66395,Unreqvited,32.0,20071.0,3,33.500000,3.500000,1.870100e+04,NaN,NaN
66396,Show Me a Dinosaur,31.0,10235.0,1,34.000000,4.500000,2.361900e+04,NaN,NaN
66397,Flovry,53.0,7478.0,1,62.000000,2.000000,5.242000e+03,NaN,NaN
66398,The Bled,32.0,33531.0,7,56.000000,8.000000,3.298930e+05,NaN,NaN


**1-4) spotify_artist 전처리: 새로운 변수 top-100 만들기**



In [89]:
#새로운 열 추가
spotify_artist["top100"]=spotify_artist["rank"]

#조건에 맞는 값을 칼럼에 추가
spotify_artist['top100']=spotify_artist['top100'].fillna("No")
spotify_artist.loc[(spotify_artist['top100']!="No"),'top100']="Yes"


**1-5) spotify_artist 전처리: 불필요한 변수와 중복되는 행 제거**



In [90]:
#불필요한 변수 제거
spotify_artist = spotify_artist.drop(columns = ['song','rank'],axis=0)

#artist를 기준으로 중복되는 행 제거
spotify_artist.drop_duplicates(['artist'])

,artist,popularity,followers,num_release,pop_mean,rel_mean,fol_mean,top100
0,Drake,98.0,61377383.0,124,81.377358,53.075472,1.100285e+07,Yes
707,Ed Sheeran,95.0,93327682.0,34,80.035714,44.035714,1.472265e+07,Yes
1068,Ariana Grande,93.0,76626333.0,94,77.650000,51.900000,1.000831e+07,Yes
1345,Bad Bunny,100.0,44516743.0,30,75.628571,23.142857,8.425305e+06,Yes
1350,The Weeknd,98.0,42275433.0,108,81.928571,53.178571,1.363645e+07,Yes
...,...,...,...,...,...,...,...,...
66395,Unreqvited,32.0,20071.0,3,33.500000,3.500000,1.870100e+04,No
66396,Show Me a Dinosaur,31.0,10235.0,1,34.000000,4.500000,2.361900e+04,No
66397,Flovry,53.0,7478.0,1,62.000000,2.000000,5.242000e+03,No
66398,The Bled,32.0,33531.0,7,56.000000,8.000000,3.298930e+05,No


### **데이터 전처리2. spotify_song 데이터 만들기**

songs_normalize.csv는 2000년부터 2019년까지 스포티파이에서 가장 인기있던 2000개의 곡에 대한 정보를 보여준다. charts 데이터와 연결해 해당 곡이 top100 차트에 오른 적이 있었는지를 판단하고자 한다. 

* popularity: 노래의 좋아요 수
* danceability
* energy
* key
* loudness
* speechiness
* acousticness
* instrumentalness
* liveness
* valence
* tempo
* genre
* top100



In [109]:
#songs_normalize.csv파일 불러오기
songs = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ITB 2022/songs_normalize.csv")

#데이터 탐색하기
display(songs.head())

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


**2-3) charts와 songs를 합친 새로운 데이터프레임 spotify_song 만들기**

In [115]:
#charts 복사하기
charts_2 = charts.copy()

#'song'을 기준으로 데이터 합치기
spotify_song = pd.merge(songs,charts_2,how='left',on='song')
display(spotify_song)

,artist_x,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,artist_y,rank
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,96.0
1,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,90.0
2,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,85.0
3,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,77.0
4,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35555,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,1,0.0395,0.192,0.002440,0.0863,0.553,120.042,hip hop,Post Malone,7.0
35556,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,1,0.0395,0.192,0.002440,0.0863,0.553,120.042,hip hop,Post Malone,8.0
35557,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,1,0.0395,0.192,0.002440,0.0863,0.553,120.042,hip hop,Post Malone,9.0
35558,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,1,0.0395,0.192,0.002440,0.0863,0.553,120.042,hip hop,Post Malone,4.0


**2-4) spotify_song 전처리: 새로운 변수 top-100 만들기**



In [116]:
#새로운 열 추가
spotify_song["top100"]=spotify_song["rank"]

#조건에 맞는 값을 칼럼에 추가
spotify_song['top100']=spotify_song['top100'].fillna("No")
spotify_song.loc[(spotify_song['top100']!="No"),'top100']="Yes"

display(spotify_song)

,artist_x,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,artist_y,rank,top100
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,...,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,96.0,Yes
1,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,...,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,90.0,Yes
2,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,...,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,85.0,Yes
3,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,...,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,77.0,Yes
4,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,...,0.0437,0.300,0.000018,0.3550,0.894,95.053,pop,Britney Spears,49.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35555,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,...,0.0395,0.192,0.002440,0.0863,0.553,120.042,hip hop,Post Malone,7.0,Yes
35556,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,...,0.0395,0.192,0.002440,0.0863,0.553,120.042,hip hop,Post Malone,8.0,Yes
35557,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,...,0.0395,0.192,0.002440,0.0863,0.553,120.042,hip hop,Post Malone,9.0,Yes
35558,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,...,0.0395,0.192,0.002440,0.0863,0.553,120.042,hip hop,Post Malone,4.0,Yes


**2-5) spotify_song 전처리: 불필요한 변수와 중복되는 행 제거**



In [117]:
#불필요한 변수 제거
spotify_song = spotify_song.drop(columns = ['artist_x','artist_y','rank','duration_ms','explicit','year','mode'],axis=0)

#중복되는 행 제거
spotify_song.drop_duplicates(['song'])

,song,popularity,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,top100
0,Oops!...I Did It Again,77,0.751,0.834,1,-5.444,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop,Yes
20,All The Small Things,79,0.434,0.897,0,-4.918,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop",Yes
39,Breathe,66,0.529,0.496,7,-9.007,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country",Yes
153,It's My Life,78,0.551,0.913,0,-4.063,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal",Yes
201,Bye Bye Bye,65,0.614,0.928,8,-4.806,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35483,Sucker,79,0.842,0.734,1,-5.065,0.0588,0.0427,0.000000,0.1060,0.952,137.958,pop,Yes
35521,Cruel Summer,78,0.552,0.702,9,-5.707,0.1570,0.1170,0.000021,0.1050,0.564,169.994,pop,Yes
35523,The Git Up,69,0.847,0.678,9,-8.635,0.1090,0.0669,0.000000,0.2740,0.811,97.984,"hip hop, country",Yes
35543,Dancing With A Stranger (with Normani),75,0.741,0.520,8,-7.513,0.0656,0.4500,0.000002,0.2220,0.347,102.998,pop,No
